In [ ]:
!pip install ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing

In [ ]:
import os
import shutil
import subprocess
import cv2
import numpy as np
from PIL import Image

# ----- Paths and config -----
image_id = "11343"
image_name = f"{image_id}.jpg"
base_dir = r"C:\Users\menna\Desktop\GP Last"
original_image = os.path.join(base_dir, "Help", image_name)

# Temp folders
processing_dir = os.path.join(base_dir, "Processing Folder", "image processing")
os.makedirs(processing_dir, exist_ok=True)
temp_image_path = os.path.join(processing_dir, image_name)

# Human parsing output
parser_output = os.path.join(base_dir, "Self-Correction-Human-Parsing", "output")
mask_path = os.path.join(parser_output, f"{image_id}.png")

# Final output
final_output_dir = os.path.join(base_dir, "masked_fashion_data_single")
os.makedirs(final_output_dir, exist_ok=True)

# ----- Copy image to processing folder -----
if not os.path.exists(temp_image_path):
    shutil.copy(original_image, temp_image_path)

# ----- Run human parsing -----
print("▶️ Running Human Parsing...")
subprocess.run([
    "python",
    os.path.join(base_dir, "Self-Correction-Human-Parsing", "simple_extractor.py"),
    "--dataset", "atr",
    "--model-restore", os.path.join(base_dir, "Self Correction Human Parsing models", "exp-schp-201908301523-atr.pth"),
    "--input-dir", processing_dir,
    "--output-dir", parser_output
], check=True)

# ----- Load image and mask -----
original = cv2.imread(temp_image_path)
mask = cv2.imread(mask_path)

if original is None or mask is None:
    print("❌ Failed to load image or mask.")
    exit()

original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
mask_rgb = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
height, width = mask_rgb.shape[:2]
total_pixels = height * width

# ----- Check color coverage -----
check_colors = [
    (0, 128, 0), (192, 128, 0), (192, 128, 128),
    (64, 128, 128), (64, 0, 128), (192, 0, 128)
]
color_mask_combined = np.zeros((height, width), dtype=np.uint8)
for color in check_colors:
    match = np.all(mask_rgb == np.array(color), axis=-1)
    color_mask_combined = np.logical_or(color_mask_combined, match)

coverage = np.sum(color_mask_combined) / total_pixels
if coverage <= 0.025:
    rgba = cv2.cvtColor(original_rgb, cv2.COLOR_RGB2RGBA)
    rgba[:, :, 3] = 255
    Image.fromarray(rgba).save(os.path.join(final_output_dir, f"{image_id}_original.png"))
    print(f"⚠️ Skipped {image_id}: <5% relevant color. Saved original.")
else:
    # ----- Category map -----
    category_color_map = {
        "Dress": {(128, 128, 128): "Dress"},
        "Topwear": {(0, 0, 128): "Topwear"},
        "Bottomwear": {(0, 128, 128): "Bottomwear"},
        "Shoes": {(64, 128, 0): "Shoes", (192, 0, 0): "Shoes"},
        "Skirt": {(128, 0, 128): "Skirt"}
    }

    # ----- Save masks per category -----
    category_found = False
    for category, color_map in category_color_map.items():
        final_mask = np.zeros(mask_rgb.shape[:2], dtype=np.uint8)
        for color in color_map:
            match = np.all(mask_rgb == np.array(color), axis=-1).astype(np.uint8) * 255
            final_mask = cv2.bitwise_or(final_mask, match)

        if np.any(final_mask):
            category_found = True
            masked = cv2.bitwise_and(original_rgb, original_rgb, mask=final_mask)
            rgba = cv2.cvtColor(masked, cv2.COLOR_RGB2RGBA)
            rgba[:, :, 3] = final_mask
            Image.fromarray(rgba).save(os.path.join(final_output_dir, f"{image_id}_{category}.png"))
            print(f"✅ Saved: {image_id}_{category}.png")

    if not category_found:
        rgba = cv2.cvtColor(original_rgb, cv2.COLOR_RGB2RGBA)
        rgba[:, :, 3] = 255
        Image.fromarray(rgba).save(os.path.join(final_output_dir, f"{image_id}_original.png"))
        print(f"⚠️ No category match. Saved original.")

# ----- Clean up -----
shutil.rmtree(os.path.join(base_dir, "Processing Folder"), ignore_errors=True)
print("🧹 Cleaned temporary processing folder.")
